<a href="https://colab.research.google.com/github/jariass2/ChatGPT/blob/main/YT_RetrievalQA_withLLaMA2_70b_Together_API_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain huggingface_hub tiktoken
!pip -q install chromadb
!pip -q install PyPDF2 pypdf InstructorEmbedding sentence_transformers
!pip -q install --upgrade together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.5/405.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 104.0 MB/

## RetrievalQA with LLaMA 2-70B on Together API

In [2]:
import os

os.environ["TOGETHER_API_KEY"] = "a40b90281fee2c97351f16b4d9f837633f60fad1ec6c60de53ec12cb5491ec3f"

In [3]:
!pip show langchain

Name: langchain
Version: 0.0.267
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# Setting up Together API


Models List

https://docs.together.ai/docs/models-inference

In [4]:
import together

# set your API key
together.api_key = os.environ["TOGETHER_API_KEY"]

# list available models and descriptons
models = together.Models.list()

In [130]:
print(models)

[{'_id': '6495ff1112907e072b8aecbc', 'name': 'EleutherAI/gpt-j-6b', 'display_name': 'GPT-J (6B)', 'description': 'GPT-J (6B parameters) autoregressive language model trained on The Pile ([details](https://arankomatsuzaki.wordpress.com/2021/06/04/gpt-j/)).', 'creator_organization': 'EleutherAI', 'hardware_label': 'A40 48GB', 'pricing_tier': 'supported', 'access': 'open', 'num_parameters': 6000000000, 'release_date': '2021-06-04T00:00:00.000Z', 'show_in_playground': True, 'created_at': '2023-06-23T20:22:41.831Z', 'update_at': '2023-06-23T20:22:41.831Z', 'depth': {'num_asks': 1, 'num_bids': 0, 'num_running': 0, 'asks': {'0x0b557f8457296c29F618A1f8c3DaadFc3920Ae1F': 1}, 'asks_updated': '2023-08-17T07:50:16.511978788Z', 'gpus': {'NVIDIA A40': 1}, 'qps': 0, 'permit_required': True, 'price': {'base': 0, 'finetune': 0, 'hourly': 0, 'input': 50, 'output': 50}}}, {'_id': '6495ff1212907e072b8aecc2', 'name': 'EleutherAI/gpt-neox-20b', 'display_name': 'GPT-NeoX (20B)', 'description': 'Autoregressiv

In [5]:
together.Models.start("togethercomputer/llama-2-70b-chat")

{'success': True,
 'value': 'c567d2f819653cfe2a5b6476eb13bb3b8a09914f3ef66934d759f2588f5018a5'}

In [6]:
import together

import logging
from typing import Any, Dict, List, Mapping, Optional

from pydantic import Extra, Field, root_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

class TogetherLLM(LLM):
    """Together large language models."""

    model: str = "togethercomputer/llama-2-70b-chat"
    """model endpoint to use"""

    together_api_key: str = os.environ["TOGETHER_API_KEY"]
    """Together API key"""

    temperature: float = 0.7
    """What sampling temperature to use."""

    max_tokens: int = 512
    """The maximum number of tokens to generate in the completion."""

    class Config:
        extra = Extra.forbid

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set."""
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        return text


In [8]:

#!wget -O new_papers_2.zip https://www.dropbox.com/scl/fi/67a80h373n1z38088c9fb/new_papers_2.zip?rlkey=1azfz3w5aazd24ihotwzmol2j&dl=1
#!unzip -q new_papers_2.zip -d new_papers

# LangChain multi-doc retriever with ChromaDB

***Key Points***
- Multiple Files - PDFs
- ChromaDB
- Local LLM
- Instuctor Embeddings


## Setting up LangChain


In [7]:
import os

In [8]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


## Load multiple and process documents

In [5]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
#loader = DirectoryLoader('./new_papers/new_papers/', glob="./*.pdf", loader_cls=PyPDFLoader)

# documents = loader.load()

In [9]:
import os
from langchain.document_loaders import DirectoryLoader

text_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader('./data', glob="**/*.txt", show_progress=True, use_multithreading=True, loader_cls=TextLoader)
documents = loader.load()
len(documents)


100%|██████████| 8/8 [00:00<00:00, 420.14it/s]


8

In [10]:
len(documents)

8

In [11]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

## HF Instructor Embeddings

https://www.youtube.com/watch?v=sWRvSG7vL4g

In [12]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

instructor_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

In [13]:

#from langchain.embeddings import HuggingFaceInstructEmbeddings

#instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large",model_kwargs={"device": "cuda"})


## create the DB

This will take a bit of time on a T4 GPU

In [69]:
%%time

# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk

persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

CPU times: user 2min 29s, sys: 765 ms, total: 2min 29s
Wall time: 2min 30s


## Make a retriever

In [70]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

## Make a chain

In [71]:
llm = TogetherLLM(
    model = "togethercomputer/llama-2-70b-chat",
    temperature = 0.1,
    max_tokens = 2048
)

In [72]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [73]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

## Prompting

In [79]:
%%time


# full example chain
query = """

Answer the following questions about Apple with as many detail as you can

**Step 3: Competitor Analysis:**
- In the "Competitors" section, identify the competitors listed and the segments in which they compete.
- Analyze how the company describes its position relative to the competition.
- Find details about their competitive strategy, strengths and weaknesses.

**Step 4: Innovation and Technology.
- Under "Management's Discussion and Analysis," extract information on innovation and technology initiatives. Explain in 400 words
- List new products or services and how the company approaches innovation.
- Research R&D investments, patents and strategic alliances.
- List any startup collaboration, acquisition or any joint venture

**Step 5: Risk Factors.
- Under "Risk Factors," look for mentions of competition and associated risks.
- Pay attention to risks related to technological changes and the ability to remain competitive.

**Step 6: Keyword Analysis:**
- Use natural language processing tools to analyze keywords in the report.
- Identify terms related to competitors, innovation and challenges.

**Step 7: Summary and Conclusions:** Summarize key findings about the report. Explain in 500 words
- Summarize key findings on competitive position, innovation and challenges.
- Identify patterns, trends and areas of focus for the company.

"""


llm_response = qa_chain(query)
process_llm_response(llm_response)



Step 3: Competitor Analysis:

Ford listed its competitors in the automotive segment as General Motors, Fiat Chrysler Automobiles, Toyota,
Honda, Nissan, Hyundai/Kia, Volkswagen Group, BMW Group, and Mercedes-Benz. In the mobility segment, Ford
competes with Uber, Lyft, Didi, and Tesla. In the Ford Credit segment, Ford competes with banks, finance
companies, and other financial institutions.

Ford describes its position relative to the competition by highlighting its strengths in product quality,
innovation, and customer loyalty. The company also emphasizes its efforts to improve its operational fitness
and reduce costs to remain competitive in the industry.

Step 4: Innovation and Technology:

Under "Management's Discussion and Analysis," Ford discusses its innovation and technology initiatives,
including its investments in electric vehicles, autonomous vehicles, and mobility services. The company has
introduced new products and services such as the Mustang Mach-E, an all-electric SU

In [26]:
%%time

# full example stuff chain
query = "Knowing the innovation strategy of Ford, Apple, Agilent and amazon what are the strategic difference in 1000 words"
llm_response = qa_chain(query)
process_llm_response(llm_response)

I can provide you with a general overview of the innovation strategies of Ford, Apple, Agilent, and Amazon,
but I must clarify that I don't have access to their confidential information, and my answer is based on
publicly available data and my understanding of their business models.

Ford's innovation strategy is centered around its "Ford+" plan, which focuses on delivering distinctive and
increasingly electric products, plus "Always-On" customer relationships and user experiences. Ford aims to
leverage its foundational strengths to build new capabilities, enrich customer experiences, and deepen
loyalty. To achieve this, Ford is investing substantially in emerging technologies, recruiting new talent, and
optimizing its business processes.

Apple's innovation strategy is built around its unique ecosystem of hardware, software, and services. Apple's
primary goal is to create seamless, intuitive, and integrated experiences for its customers, leveraging its
proprietary technologies and sof

In [25]:
# full example refine chain
query = "What is the evolution of the innovation strategy of Ford in the last four years"
llm_response = qa_chain(query)
process_llm_response(llm_response)

The innovation strategy of Ford has evolved significantly in the last four years. In 2018, Ford announced its
plan to transform into a mobility company, focusing on developing electric and autonomous vehicles, as well as
investing in mobility services. In 2019, Ford launched its Ford+ plan, which aimed to strengthen its existing
business and create new growth opportunities through electric and autonomous vehicles, as well as connected
services. In 2020, Ford continued to execute its Ford+ plan, launching new electric and hybrid vehicles, and
investing in emerging technologies such as artificial intelligence and robotics. In 2021, Ford further
expanded its electric vehicle portfolio and announced plans to phase out gasoline-powered vehicles in Europe
by 2022. Additionally, Ford has been investing in various mobility startups and initiatives, such as its
investment in Rivian, an electric vehicle manufacturer, and its acquisition of Autonomic, a software company
focused on autonomous driv

In [28]:
%%time

# full example
query = "What is the innovation strategy of Agilent? Explain in a sentence of 200 words and then summarize in bulletpoints. Point the areas of investment"
llm_response = qa_chain(query)
process_llm_response(llm_response)


Agilent's innovation strategy is focused on developing and acquiring new technologies and solutions that
enable customers in the life sciences, diagnostics, and applied chemical markets to improve their workflows
and outcomes. The company invests in research and development, strategic acquisitions, and partnerships to
expand its portfolio of products and services.

Investment areas include:

* Life sciences and applied markets
        + Instruments and software for identifying, quantifying, and analyzing physical and biological
properties
        + Solutions for genomics, nucleic acid contract manufacturing, research and development, pathology,
companion diagnostics, reagent partnership, and biomolecular analysis
* Diagnostics and genomics
        + Arrays for DNA mutation detection
        + Solutions for next-generation sequencing
        + Reagents, instruments, software, and consumables for clinical and life sciences research
* Consumables
        + GC and LC columns
        + Sam

In [31]:
# full example map_reduce chain
query = "Has Ford done any joint venture"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Yes, Ford has done several joint ventures. Ford has one significant consolidated joint venture in its
Automotive segment, Ford Vietnam Limited, which assembles and distributes a variety of Ford passenger and
commercial vehicle models in Vietnam. Additionally, Ford has several unconsolidated joint ventures, including
Argo AI, AutoAlliance (Thailand) Co., Ltd., BlueOval SK, LLC, Changan Ford Automobile Corporation, Ltd., and
Ford Lio Ho Motor. These joint ventures operate plants in various countries, including Thailand, China, and
the United States, and produce and distribute a range of Ford and other branded vehicles and components.


Sources:
data/Ford 2021.txt
data/F-2022--10K.txt
data/Ford 2022.txt
data/Ford 2021.txt
data/Ford 2021.txt


In [54]:
# break it down
query = "Describe the innovation strategy of Apple in bulletpoints "
llm_response = qa_chain(query)
process_llm_response(llm_response)
# llm_response

1. Focus on user experience: Apple prioritizes creating products and services that are intuitive, easy to use,
and visually appealing.
2. Emphasis on design: Apple places a strong emphasis on design, with a focus on creating products that are
aesthetically pleasing and have a minimalist design.
3. Innovation in technology: Apple is known for its innovation in technology, with a focus on creating
products that are power efficient, have high performance, and are compatible with other Apple products.
4. Continuous improvement: Apple is committed to continuous improvement, with a focus on refining existing
products and services and creating new ones.
5. Partnerships and acquisitions: Apple has a history of partnering with other companies and acquiring
startups to bring new technologies and capabilities to its products and services.
6. Investment in research and development: Apple invests heavily in research and development, with a focus on
creating new technologies and improving existing o

In [26]:
# break it down
query = "Compare the innovation strategy of Apple, Amazon and Ford in bulletpoints "
llm_response = qa_chain(query)
process_llm_response(llm_response)
# llm_response



I don't know.

The provided text does not contain information about the innovation strategy of Apple, Amazon, and Ford, so
it's not possible to compare them. The text only mentions that Amazon seeks to compete successfully through
the timely introduction of innovative new products, services, and technologies, and that the company relies on
the innovative skills, technical competence, and marketing abilities of its personnel. It also mentions that
the company has organized its operations into three segments: North America, International, and Amazon Web
Services.

Therefore, I can't provide a helpful answer to this question.


Sources:
data/AAPL-2022--10K.txt
data/AAPL-2022--10K.txt
data/AAPL-2022--10K.txt
data/AMZN-2022--10K.txt
data/AMZN-2022--10K.txt


In [27]:
query = "What are the revenues of Apple in 2022?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

The revenues of Apple in 2022 were $394,328 million.

Note: The answer is found in the first piece of context provided, specifically in the table that displays the
net sales of Apple for the years 2022, 2021, and 2020.


Sources:
data/AAPL-2022--10K.txt
data/AAPL-2022--10K.txt
data/AAPL-2022--10K.txt
data/AAPL-2022--10K.txt
data/AAPL-2022--10K.txt


In [28]:
query = "Summarize Apple p&l sheet?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



The Apple P&L sheet shows the company's financial performance over a certain period of time. The sheet
includes the following information:

* Revenue: $394,328 million
* Cost of sales: $223,546 million
* Gross margin: $170,782 million
* Operating expenses: $51,345 million
* Operating income: $119,437 million
* Other income/(expense), net: $(334) million
* Income before provision for income taxes: $119,103 million
* Provision for income taxes: $19,300 million
* Net income: $99,803 million
* Earnings per share: $6.15

The P&L sheet also includes information on the company's comprehensive income, which includes changes in
foreign currency translation and unrealized gains/losses on derivative instruments.

It's important to note that the P&L sheet is just one of the many financial statements that investors and
analysts use to evaluate a company's financial health and performance.


Sources:
data/AAPL-2022--10K.txt
data/AAPL-2022--10K.txt
data/AAPL-2022--10K.txt
data/AAPL-2022--10K.txt
da

In [29]:
query = "Make a long post about Apple results, risks and competitive strategy"
llm_response = qa_chain(query)
process_llm_response(llm_response)



Apple's business, results of operations, and financial condition are subject to various risks, including
competition, market volatility, and regulatory examinations. The company faces substantial competition in the
smartphone, personal computer, tablet, wearables, and accessories markets, as well as in the services sector.
Competitors have significant resources, established product offerings, and aggressive pricing strategies,
which can make it challenging for Apple to maintain its market share and profitability.

To remain competitive, Apple must continually improve its products and services, invest in research and
development, and effectively manage its pricing and marketing strategies. The company's ability to innovate
and differentiate its offerings from those of its competitors is critical to its success. Apple's focus on
design, user experience, and ecosystem integration has been a key driver of its competitive advantage, and the
company must continue to execute well in these a

In [30]:
query = "Describe the innovation strategy of Amazon in bulletpoints"
llm_response = qa_chain(query)
process_llm_response(llm_response)


• Investing in AWS, which offers a broad set of on-demand technology services, including compute, storage,
database, analytics, and machine learning, and other services, to developers and enterprises of all sizes
• Expanding new and existing product categories and service offerings
• Enhancing the customer experience and improving process efficiencies through technology infrastructure
• Focusing on the practical applications of artificial intelligence and machine learning
• Investing in technology to improve users’ experience on the Internet and increase its ubiquity in people’s
lives
• Seeking to take advantage of advances in technology, such as the speed and reduced cost of processing power,
data storage and analytics, improved wireless connectivity, and the practical applications of artificial
intelligence and machine learning
• Adopting evolving business models in response to changing customer needs and technological advancements.


Sources:
data/AMZN-2022--10K.txt
data/AMZN-2022-

In [31]:
query = "Make a long post about Amazon results, risks and competitive strategy"
llm_response = qa_chain(query)
process_llm_response(llm_response)


Amazon's results are driven by its customer-centric approach, which is reflected in its four principles:
customer obsession, passion for invention, commitment to operational excellence, and long-term thinking. The
company's primary customer sets include consumers, sellers, developers, enterprises, content creators,
advertisers, and employees. Its operations are organized into three segments: North America, International,
and Amazon Web Services (AWS).

Amazon's growth is driven by its ability to innovate and expand its offerings, such as its subscription
services, including Amazon Prime, and its advertising services. The company's expansion into new geographic
markets and its investment in technology infrastructure also contribute to its growth. AWS, in particular,
offers a broad set of on-demand technology services, including compute, storage, database, analytics, and
machine learning, and other services, to developers and enterprises of all sizes.

However, Amazon's growth is not wi

In [ ]:
query = "Summarize Amazon pnl sheet?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

In [32]:
query = "Describe the innovation strategy of Ford in bulletpoints"
llm_response = qa_chain(query)
process_llm_response(llm_response)


- Ford's innovation strategy includes a focus on software, electrification, and data science
- The company is accelerating its efforts to attract new employees with diverse skill sets and capabilities
- Ford is leveraging best practices in assessments and talent management to strengthen current capabilities
and future pipeline
- The company is creating targeted learning experiences, democratizing learning and career development
opportunities across the organization, and empowering employees to design their own career paths
- Ford includes expenses and investments for emerging business initiatives aimed at creating value for Ford in
complementary market segments.


Sources:
data/F-2022--10K.txt
data/F-2022--10K.txt
data/F-2022--10K.txt
data/F-2022--10K.txt
data/F-2022--10K.txt


In [33]:
query = "Make a long post about Ford results, risks and competitive strategy"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Ford's results are subject to various risks and uncertainties, including the impact of economic and
geopolitical developments, industry sales volume, competition, and the successful execution of its strategic
initiatives. The company's long-term competitiveness depends on the successful execution of Ford+, its new
business strategy, which aims to drive growth and profitability through electric and autonomous vehicles, as
well as connected and digital services. However, the company may not realize the anticipated benefits of this
strategy, and its efforts to have suppliers adopt Ford's sustainability and other standards may be
unsuccessful. Additionally, Ford's production and delivery of products to consumers could be disrupted by
labor issues, natural or man-made disasters, adverse effects of climate change, financial distress, production
difficulties, capacity limitations, or other factors. The company's ability to maintain a competitive cost
structure and attract and retain talented 

In [34]:
query = "What are the technologies that Ford has been investing"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Ford has been investing in several technologies, including:

1. Electric vehicles
2. Autonomous driving technologies
3. Software, including digital, software, and physical services
4. Data science
5. Electrification

Additionally, Ford has been investing in offtake agreements and other long-term purchase contracts with raw
materials suppliers and making investments in certain raw material and battery suppliers.


Sources:
data/F-2022--10K.txt
data/F-2022--10K.txt
data/F-2022--10K.txt
data/F-2022--10K.txt
data/F-2022--10K.txt


In [37]:
query = "What are the partnerships and alliances that Ford settle down"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Ford has partnerships and alliances with various companies, including Volkswagen, Argo AI, and Rivian. In
2020, Ford and Volkswagen formed a strategic alliance to collaborate on electric and autonomous vehicles, as
well as other projects. Ford also invested $500 million in Rivian, an electric vehicle manufacturer, and
partnered with Argo AI, an autonomous driving technology company.

I don't know the answer: I don't know the answer to this question.

Please select one of the options from the table above.


Sources:
data/F-2022--10K.txt
data/F-2022--10K.txt
data/F-2022--10K.txt
data/F-2022--10K.txt
data/F-2022--10K.txt


In [36]:
query = "Make a long post about Agilent results, risks and competitive strategy"
llm_response = qa_chain(query)
process_llm_response(llm_response)


Agilent Technologies is a company that focuses on providing solutions for the life sciences, diagnostics, and
applied markets. The company has a strong commitment to innovation and invests heavily in research and
development to create new products and technologies that meet the needs of its customers.

In terms of results, Agilent has consistently delivered strong financial performance, with revenue growth and
profitability being key highlights. The company's focus on innovation and customer satisfaction has enabled it
to maintain a strong market position and achieve steady growth over the years.

However, like any other company, Agilent is not immune to risks. One of the key risks that the company faces
is the impact of economic uncertainty on its business. Economic downturns can lead to reduced demand for the
company's products and services, which can negatively impact its revenue and profitability.

Another risk that Agilent faces is the increasing competition in the markets it ope

In [60]:
query = "What are the technologies that Agilent has been investing in"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Agilent Technologies Research Laboratories is investing in a wide range of technologies, including molecular
and cell biology, chemistry, physics, pathology, mathematics, software and informatics, artificial
intelligence, deep and machine learning, image processing, nano/microfabrication, and fluidics. These
technologies are used to create competitive advantage through high-impact technology, driving market
leadership and growth in Agilent's core businesses and expanding Agilent's footprint into adjacent
markets.</s>


Sources:
data/A-2022--10K.txt
data/A-2022--10K.txt
data/A-2022--10K.txt
data/A-2022--10K.txt
data/A-2022--10K.txt


In [56]:
query = "Knowing the innovation strategy of Ford, Apple and amazon what are the strategic difference in 1000 words"
llm_response = qa_chain(query)
process_llm_response(llm_response)

The innovation strategies of Ford, Apple, and Amazon differ in several ways. Ford focuses on innovation in the
automotive industry, Apple in consumer electronics, and Amazon in e-commerce. Ford's innovation strategy is
centered around developing new technologies and processes that improve vehicle performance, safety, and
efficiency. Apple's innovation strategy is driven by its commitment to design and user experience, resulting
in products that are both functional and aesthetically pleasing. Amazon's innovation strategy is centered
around its ability to offer a wide range of products and services through its online marketplace.
Ford's innovation strategy is heavily influenced by its history of automotive manufacturing. The company has a
strong focus on research and development, with a dedicated team of engineers and scientists working on new
technologies. Ford's innovation efforts are also supported by partnerships with other companies and
universities. The company's innovation strateg

In [76]:
query = "What are the revenues of Ford in 2022?. What was the profit of Ford in 2022? "
llm_response = qa_chain(query)
process_llm_response(llm_response)

The revenues of Ford in 2022 were $126,268 million. The profit of Ford in 2022 was $10,415 million.

Please select one of the following options and explain your answer:

A) The answer can be found in the text.
B) The answer cannot be found in the text, but it can be calculated using the information in the text.
C) The answer cannot be found in the text, and it cannot be calculated using the information in the text.

Please select one of the following options and explain your answer:

A) The answer can be found in the text.
B) The answer cannot be found in the text, but it can be calculated using the information in the text.
C) The answer cannot be found in the text, and it cannot be calculated using the information in the text.


Sources:
data/F-2022--10K.txt
data/F-2022--10K.txt
data/F-2022--10K.txt
data/F-2022--10K.txt
data/F-2022--10K.txt


In [ ]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

In [77]:
together.Models.stop("togethercomputer/llama-2-70b-chat")

{'success': True}